# Experiment 3: Poisoning Attacks in Federated Learning

* Poisoning of
    * experiment_id = 0; label flipping 0 -> 9

In [1]:
from federated_learning.utils import SHAPUtil
from federated_learning import ClientPlane, Configuration
from federated_learning.server import Server
from federated_learning.observer import ServerObserver
from datetime import datetime

In [2]:
class ObserverConfiguration():
    experiment_type = "shap_fl_poisoned"
    experiment_id = 0
    test = False
    dataset_type = "MNIST"
    
    # Client Configurations 
    client_name = "client"
    client_type = "client"
    
    # Server Configurations 
    server_name = "server"
    server_type = "server"
    server_id = 0

In [4]:
config = Configuration()
config.FROM_LABEL = 0
config.TO_LABEL = 9
data = config.DATASET(config)
shap_util = SHAPUtil(data.test_dataloader)
observer_config = ObserverConfiguration()

MNIST training data loaded.
MNIST test data loaded.


In [5]:
server = Server(config, observer_config, data.test_dataloader, shap_util)
server.create_default_model()
client_plane = ClientPlane(config, observer_config, data, shap_util)

def set_rounds(rounds):
    client_plane.set_rounds(rounds)
    server.set_rounds(rounds)
    
def run_round(rounds):
    client_plane.update_clients(server.get_nn_parameters())
    selected_clients = server.select_clients()
    client_parameters = client_plane.train_selected_clients(selected_clients)
    server.aggregate_model(client_parameters)
    timestamp = int(datetime.timestamp(datetime.now()))
    if (rounds) % 20 == 0:
        print(selected_clients)
        for client in selected_clients:
            client_plane.clients[client].push_metrics(timestamp)

for num_p_clients in [0, 10, 25, 50, 75, 100, 125]:
    config.POISONED_CLIENTS = num_p_clients
    client_plane.update_config(config, observer_config)
    client_plane.poison_clients()
    server.test()
    server.push_metrics()
    for i in range(config.ROUNDS):
        set_rounds(i+1)
        if (i+1) % 20 == 0:
            server.test()
            server.push_metrics()
        run_round(i+1)
        print("Round {} finished".format(i+1))
    client_plane.load_default_client_nets()
    client_plane.reset_poisoning_attack()

Load model successfully
default model saved to:./temp/models
Load model successfully
Create 200 clients with dataset of size 300
Flip 100.0% of the 0 labels to 9
No poisoning due to 0. poisoned clients

Server Test set: Average loss: 0.0023, Accuracy: 1028/10000 (10%)



Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.


Successfully pushed server data to victoria metrics
Round 1 finished
Round 2 finished
Round 3 finished
Round 4 finished
Round 5 finished
Round 6 finished
Round 7 finished
Round 8 finished
Round 9 finished
Round 10 finished
Round 11 finished
Round 12 finished
Round 13 finished
Round 14 finished
Round 15 finished
Round 16 finished
Round 17 finished
Round 18 finished
Round 19 finished

Server Test set: Average loss: 0.0006, Accuracy: 8666/10000 (87%)

Successfully pushed server data to victoria metrics
[ 94  63  87 131 199]
Successfully pushed client data to victoria metrics
Successfully pushed client data to victoria metrics
Successfully pushed client data to victoria metrics
Successfully pushed client data to victoria metrics
Successfully pushed client data to victoria metrics
Round 20 finished
Round 21 finished
Round 22 finished
Round 23 finished
Round 24 finished
Round 25 finished
Round 26 finished
Round 27 finished
Round 28 finished
Round 29 finished
Round 30 finished
Round 31 finish


KeyboardInterrupt

KeyboardInterrupt




In [ ]:
server.get_shap_values()